In [1]:
import pandas as pd
import numpy as np
import requests
from pandas.io.html import read_html

In [5]:
# Cria lista de periodos para avaliação.
begining_year = 2017
begining_bimester = 1

semester = [begining_year,begining_bimester]

max_year = 2022
max_bimester = 1
max_semester = [max_year,max_bimester]

semesters = []
while semester != max_semester:
    semesters.append(f'{semester[0]}{semester[1]}')
    if   semester[1] == 1: 
        semester[1] = 2
    elif semester[1] == 2: 
        semester[1] = 1 
        semester[0] +=1
semesters.append(f'{max_semester[0]}{max_semester[1]}')
print(semesters)

['20171', '20172', '20181', '20182', '20191', '20192', '20201', '20202', '20211', '20212', '20221']


In [6]:
#Faz downloads das tabelas das páginas do portal
#A engenharia elétrica possui duas páginas, devido à quantidade de disciplinas.

page  = 'https://app.uff.br/transparencia/disciplinas_que_mais_reprovam?utf8=true&curso=9&periodo=20171'
page2 = 'https://app.uff.br/transparencia/disciplinas_que_mais_reprovam?curso=9&page=2&periodo=20171&utf8=true'


#Uma tabela por período
tables_uff = []

#Unifica em uma tabela as duas páginas
for semester in semesters:
    page = page.replace(page[89::],semester)
    page2 = page2.replace(page2[86:91],semester)
    table_uff_page1 = read_html(page,  attrs={"class":"table table-striped table-bordered table-hover"})
    table_uff_page2 = read_html(page2,  attrs={"class":"table table-striped table-bordered table-hover"})
    table_uff=table_uff_page1[0].append(table_uff_page2[0],ignore_index=True)
    tables_uff.append(table_uff)

#Imprime final das listas
print(tables_uff[1].tail(3))
print(tables_uff[2].tail(3))

      Código                               Disciplina  Reprovações
50  MND00025  ESTAGIO SUPERVISIONADO NUTRICAO CLINICA            1
51  MNS00042                      ALIMENTAÇÃO ESCOLAR            1
52  MBO00023                 MICROSCÓPIA DE ALIMENTOS            1
      Código                               Disciplina  Reprovações
53  MND00025  ESTAGIO SUPERVISIONADO NUTRICAO CLINICA            1
54  MIP00088                        PARASITOLOGIA III            1
55  MNS00042                      ALIMENTAÇÃO ESCOLAR            1


In [7]:
#Organiza conjunto de nomes de disciplinas 
set_discipline = set([])
# Como cada periodo possui um número diferente de disciplinas 
# utilizei a estrutura de conjuntos para agrupar todas as disciplinas das tabelas sem permitir repetição

for table in tables_uff:
    set_discipline.update(table['Disciplina'])

# Lista de disciplinas
#print(set_discipline)
# Quantidade de disciplinas
print(f'total de disciplinas: {len(set_discipline)}')

#Cria dicionário {chave: nome_da_disciplina: nº_de_reprovações}
#nº de reprovações inicializado com zero
reprovations={}
for discipline in set_discipline:
    reprovations[discipline] = np.zeros(len(tables_uff),np.int16)

# Completa o dicionário
j=0
for table in tables_uff:
        for i in range(len(table)):
                discipline = table['Disciplina'][i]
                n_reprovations = table['Reprovações'][i]
                reprovations[discipline][j] = n_reprovations
        j=j+1


total de disciplinas: 90


In [ ]:
#Constroi tabela do excel(.csv) Disciplina; nº de reprovações por semestre;
import csv
with open('Reprovações Nutrição.csv', 'w') as f:

    #Cabeçalho
    f.write('Disciplina')
    for semester in semesters:
        f.write(f';{semester[:4]}.{semester[4]}')
    f.write('\n')

    #Dados
    for discipline_name in reprovations.keys():
        f.write(discipline_name)
        for reprovation in reprovations[discipline_name]: 
            f.write(f';{reprovation}')
        f.write('\n')